In [1]:
import appmode
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize
from sklearn.pipeline import make_pipeline
# Loading relevant libraries
from gensim.models import LdaModel, LdaMulticore
from gensim.models.phrases import Phrases, Phraser
from gensim import corpora
import re
import pyLDAvis.gensim
# import logging
import gensim
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
# logging.root.setLevel(level=logging.INFO)
# stop_words = stopwords.words('english')
# stop_words = stop_words + ['com', 'edu', 'would', 'could', 'also', 'le', 'thank_you', 'u', 'etc']

%matplotlib inline 

In [2]:
from ipywidgets import Button, Layout, Box, interactive, fixed, HBox, Label
import ipywidgets as widgets

In [3]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import nltk

In [4]:
import warnings
warnings.filterwarnings("ignore",category=PendingDeprecationWarning)
warnings.filterwarnings("ignore",category=DeprecationWarning)
warnings.filterwarnings(action='once')

In [5]:
answer_df = pd.read_csv('N:\\ORP_accountability\\projects\\Andrew\\NLP with TES\\answered_q13_df.csv')
# answer_df.head()

In [6]:
# answer_df.YrsExpr18.sort_values().unique()

In [7]:
def label_yrs_exp (row):
    yrs_exp = row['YrsExpr18']
    if yrs_exp > 25:
          return '25+'
    elif yrs_exp > 15:
        return '15-25'
    elif yrs_exp > 5:
        return '5-15'
    return '<5'

answer_df['yrs_exp_cat'] = answer_df.apply(label_yrs_exp, axis=1)

In [8]:
# answer_df.head()

In [9]:
def make_trigrams_mod(texts, t_mod, b_mod):
    return [t_mod[b_mod[doc]] for doc in texts]

In [10]:
# Outputs list of lists with trigrams for each answer in a list
def preprocess_responses(text_list, bigram_min_count = 3, bigram_threshold = 8, trigram_min_count = 2, trigram_threshold = 8):
    # Make all words lowercase and remove punctuation
    wnl = nltk.WordNetLemmatizer()
    lower_no_punctuation = [[text for text in word_tokenize(doc.lower()) if text.isalpha()] for doc in text_list]
    # Establish stop words
    # Needs stopwords from nltk.corpus
    stop_words = stopwords.words('english')
    stop_words = stop_words + ['would', 'could', 'also', 'le', 'thank_you', 'u', 'etc', 't',
                              'the', 'and', 'are', 'of', 'for', 'that']
    # remove stop words
    text_no_stops = [[text for text in doc if text not in stop_words] for doc in lower_no_punctuation]
    # Lemmatize
    text_lemmatized = [[wnl.lemmatize(text) for text in doc] for doc in text_no_stops]
    # models for trigrams and bigrams
    bigram_topics = Phrases(text_lemmatized, min_count = bigram_min_count, threshold = bigram_threshold)
    trigram_topics = Phrases(bigram_topics[text_lemmatized], min_count = trigram_min_count, threshold = trigram_threshold)
    # Phraser for better performance
    bigram_mod = Phraser(bigram_topics)
    trigram_mod = Phraser(trigram_topics)
    # Make trigrams (also forms bigrams in the process)
    topic_trigrams = make_trigrams_mod(text_lemmatized, trigram_mod, bigram_mod)
    return topic_trigrams
    

In [11]:
def make_lda_topic_model(preprocessed_list, num_topics = 10, chunksize = 200, passes = 10):
    # create dictionary
    id2word = corpora.Dictionary(preprocessed_list)
    # Create Corpus
    texts = preprocessed_list
    corpus = [id2word.doc2bow(text) for text in texts] # Term Document Frequency
    # Created model with gensim LDA model
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=chunksize,
                                           passes=passes,
                                           alpha='auto',
                                           per_word_topics=True)
    return id2word, corpus, lda_model

In [12]:
def make_topic_visual(model, corpus, dictionary):
    lda_display = pyLDAvis.gensim.prepare(model, corpus, dictionary, sort_topics=False)
    return pyLDAvis.display(lda_display)

In [13]:
def total_lda_vis(response_list, 
                  bigram_min_count = 3, bigram_threshold = 8, 
                  trigram_min_count = 2, trigram_threshold = 3,
                  num_topics = 7, topic_chunksize = 200, passes = 10
                 ):
    trigram_list = preprocess_responses(response_list, 
                                        bigram_min_count = bigram_min_count, bigram_threshold = bigram_threshold, 
                                        trigram_min_count = trigram_min_count, trigram_threshold = trigram_threshold)
    dictionary, corpus, lda_model = make_lda_topic_model(trigram_list, 
                                                         num_topics = num_topics, chunksize = topic_chunksize, passes = passes)
    return make_topic_visual(lda_model, corpus, dictionary)

In [14]:
def subset_df_list(df, district, gender, yrs_exp_cat, role_compass, topics):
    n_df = df
    if district != 'Any':
        n_df = n_df.loc[n_df['district_name'] == district]
    if gender != 'Any':
        n_df = n_df.loc[n_df['Gender'] == gender]
    if yrs_exp_cat!= 'Any':
        n_df = n_df.loc[n_df['yrs_exp_cat'] == yrs_exp_cat]
    if role_compass != 'Any':
        n_df = n_df.loc[n_df['Role_Compass'] == role_compass]
    answer_list = n_df.Q13.tolist()
    print('First 5 Comments: ')
    display(answer_list[:5])
    display(total_lda_vis(answer_list, num_topics = topics))
    return answer_list

In [15]:
# subset_df_list(answer_df, 'Montgomery County', 'F', '25+', 'Any')

In [16]:
# Converting the column of the DF with answers into a list of answers
open_ended_answers = answer_df.loc[answer_df['answered_q13'] == 1,:].Q13.tolist()

In [17]:
def find_similar_responses(base_text,  answer_list, num_similar = 10, num_topics = 6):
    if base_text in answer_list:
        answer_list = answer_list
    else:
        answer_list = [base_text] + answer_list
    tfidf = TfidfVectorizer(stop_words=stopwords.words('english'), max_df = 0.80)
    responses = tfidf.fit_transform(answer_list)
    model = NMF(n_components=num_topics)
    nmf_features = model.fit_transform(responses)
    norm_features = normalize(nmf_features)
    normalized_df = pd.DataFrame(norm_features)
    response = normalized_df.iloc[0]
    similar = normalized_df.dot(response)
    similar_ind_list = similar.sort_values(ascending = False).index.tolist()[1:num_similar+1]
    display(base_text)
    out = [answer_list[ind] for ind in similar_ind_list]
    print('Results: ')
    display(out)
    # print(response)
    return out

In [18]:
# Find 5 responses similar to the 2nd comment
# 2nd comment displayed 1st and the similar comments follow
# find_similar_responses(1, open_ended_answers, num_similar = 5)

In [19]:
# answer_df.Role_Compass.sort_values().unique().tolist()

In [20]:
role_list = ['Any'] + answer_df.Role_Compass.sort_values().unique().tolist()

In [21]:
dist_list = ['Any'] + answer_df.district_name.sort_values().unique().tolist()

In [22]:
# dist_list[:5]

In [23]:
# sub_answers = interactive(subset_df_list, df = fixed(answer_df), district = dist_list, gender = ['Any', 'M', 'F'],
#                 yrs_exp_cat = ['Any', '25+', '15-25', '5-15', '<5'], role_compass = role_list, topics = range(3,24))

## Topics Based on Desired Characteristics

In [24]:
# display(sub_answers)

In [25]:
# .result returns the output of the function called in interactive
# sub_answers.result

In [26]:
# similar_display = interactive(find_similar_responses, comment_index = range(0,len(sub_answers.result)), answer_list = fixed(sub_answers.result), 
#             num_similar = range(1,31), num_topics = fixed(10))

# similar_display = interactive(find_similar_responses, base_text = 'Enter Comment Here', answer_list = fixed(sub_answers.result), 
           #  num_similar = range(1,31), num_topics = fixed(sub_answers.children[4].value))

In [27]:
# display(similar_display)

In [28]:
years_exp_range = widgets.IntRangeSlider(
    value=[1, 30],
    min=1,
    max=58,
    step=1,
)
years_range_display = HBox([Label('Years Experience: '), years_exp_range])

In [29]:
# years_exp_range


In [30]:
# n_widg = HBox([Label('Years Experience: '), years_exp_range])

In [31]:
# years_exp_range.value[0]

In [32]:
# n_widg

In [33]:
#years_exp_range.observe(update_range, 'value')

In [34]:
def update_range(*args):
    n_range = years_exp_range.value
    #print(n_range[0])

In [35]:
# sub_answers = interactive(subset_df_list, df = fixed(answer_df), district = dist_list, gender = ['Any', 'M', 'F'],
#                 yrs_exp_cat = ['Any', '25+', '15-25', '5-15', '<5'], role_compass = role_list, topics = range(3,24))

# Widgets
years_exp_range = widgets.IntRangeSlider(
    value=[1, 30],
    min=1,
    max=58,
    step=1,
)
years_range_display = HBox([Label('Years Experience: '), years_exp_range])

dist_dropdown = widgets.Dropdown(
    options= dist_list,
    value='Any',
    description='District:'
)

gender_radio = widgets.RadioButtons(
    options= ['Any', 'Female', 'Male'],
    value='Any',
    description='Gender:'
)

topics_slider = widgets.IntSlider(
    value=7,
    min=2,
    max=25,
    step=1
)
topics_display = HBox([Label('Number of Topics: '), topics_slider])

role_radio = widgets.RadioButtons(
    options= role_list[:len(role_list)-1],
    value='Any',
    description='Role:'
)




In [36]:
def topic_model_vis(df, district, gender, yrs_exp, role_compass, topics):
    min_yr = yrs_exp[0]
    max_yr = yrs_exp[1]
    if gender == 'Male':
        gender = 'M'
    elif gender == 'Female':
        gender = 'F'
    n_df = df
    if district != 'Any':
        n_df = n_df.loc[n_df['district_name'] == district]
    if gender != 'Any':
        n_df = n_df.loc[n_df['Gender'] == gender]
    n_df = n_df.loc[(n_df['YrsExpr18'] >= min_yr) & (n_df['YrsExpr18'] <= max_yr)]
    if role_compass != 'Any':
        n_df = n_df.loc[n_df['Role_Compass'] == role_compass]
    answer_list = n_df.Q13.tolist()
    print('First 5 Comments: ')
    display(answer_list[:5])
    display(total_lda_vis(answer_list, num_topics = topics))
    return answer_list

In [37]:
topic_model = interactive(topic_model_vis, df = fixed(answer_df), district = dist_dropdown, gender = gender_radio,
                 yrs_exp = years_exp_range, role_compass = role_radio, topics = topics_slider)

In [38]:
display(topic_model)

interactive(children=(Dropdown(description='District:', options=('Any', 'Achievement School District', 'Alamo'…

In [39]:
# similar_display = interactive(find_similar_responses, base_text = 'Enter Comment Here', answer_list = fixed(sub_answers.result), 
#             num_similar = range(1,31), num_topics = fixed(sub_answers.children[4].value))

# Widgets for finding similar comments

text_area = widgets.Textarea(
    placeholder='Enter Text',
    description='Base Comment:',
    disabled=False
)

dist_dropdown_similar = widgets.RadioButtons(
    options= [dist_dropdown.value],
    description='District:'
)

gender_dropdown_similar = widgets.RadioButtons(
    options= [gender_radio.value],
    description='Gender:'
)

yrs_min_dropdown_similar = widgets.RadioButtons(
    options= [years_exp_range.value[0]],
    description='Minimum Years Experience:'
)

yrs_max_dropdown_similar = widgets.RadioButtons(
    options= [years_exp_range.value[1]],
    description='Maximum Years Experience:'
)

role_dropdown_similar = widgets.RadioButtons(
    options= [role_radio.value],
    description='Role:'
)

num_topics_dropdown_similar = widgets.RadioButtons(
    options= [topics_slider.value],
    description='Number of Topics:'
)

num_similar_slider = widgets.IntSlider(
    min=2,
    max=25,
    step=1,
    description = 'Number Similar:'
)


In [40]:
# dist_dropdown.value

In [41]:
def update_similar_args(*args):
    dist_dropdown_similar.options = [dist_dropdown.value]
    gender_dropdown_similar.options = [gender_radio.value]
    yrs_min_dropdown_similar.options = [years_exp_range.value[0]]
    yrs_max_dropdown_similar.options = [years_exp_range.value[1]]
    role_dropdown_similar.options = [role_radio.value]
    num_topics_dropdown_similar.options = [topics_slider.value]

In [42]:
# Observe variables to update from first widget
dist_dropdown.observe(update_similar_args, 'value')
gender_radio.observe(update_similar_args, 'value')
years_exp_range.observe(update_similar_args, 'value')
role_radio.observe(update_similar_args, 'value')
topics_slider.observe(update_similar_args, 'value')


In [43]:
def show_similar_responses(df, base_text, district, gender, yrs_exp_min, yrs_exp_max, role_compass, num_similar = 10, num_topics = 6):
    n_df = df
    if district != 'Any':
        n_df = n_df.loc[n_df['district_name'] == district]
    if gender != 'Any':
        n_df = n_df.loc[n_df['Gender'] == gender]
    n_df = n_df.loc[(n_df['YrsExpr18'] >= yrs_exp_min) & (n_df['YrsExpr18'] <= yrs_exp_max)]
    if role_compass != 'Any':
        n_df = n_df.loc[n_df['Role_Compass'] == role_compass]
        
    answer_list = n_df.Q13.tolist()
    
    if base_text in answer_list:
        answer_list = answer_list
    else:
        answer_list = [base_text] + answer_list
    tfidf = TfidfVectorizer(stop_words=stopwords.words('english'), max_df = 0.80)
    responses = tfidf.fit_transform(answer_list)
    model = NMF(n_components=num_topics)
    nmf_features = model.fit_transform(responses)
    norm_features = normalize(nmf_features)
    normalized_df = pd.DataFrame(norm_features)
    response = normalized_df.iloc[0]
    similar = normalized_df.dot(response)
    similar_ind_list = similar.sort_values(ascending = False).index.tolist()[1:num_similar+1]
    display(base_text)
    out = [answer_list[ind] for ind in similar_ind_list]
    print('Results: ')
    display(out)
    # print(response)
    return out

## Find Similar Comments
- Input comment text and the most similar comments from the preceding list will be returned

In [44]:
# similar_comments = interactive(show_similar_responses, base_text = text_area, answer_list = fixed(update_list()), 
#             num_similar = num_similar_slider, num_topics = fixed(7))

similar_comments = interactive(show_similar_responses, df = fixed(answer_df), district = dist_dropdown_similar, 
                               gender = gender_dropdown_similar, yrs_exp_min = yrs_min_dropdown_similar, yrs_exp_max = yrs_max_dropdown_similar,
                               role_compass = role_dropdown_similar, num_similar = num_similar_slider, base_text = text_area, 
                               num_topics = num_topics_dropdown_similar)

In [45]:
display(similar_comments)

interactive(children=(Textarea(value='', description='Base Comment:', placeholder='Enter Text'), RadioButtons(…